In [36]:
import pandas as pd
import csv
import random
import numpy as np

In [15]:
''' randomly sample file '''
n = sum(1 for line in open("sgRNA_design_9606_GRCh38_enAsCas12a_CRISPRko_NCBI_20200615.txt")) - 1
s = 10000
skip = sorted(random.sample(range(1,n+1),n-s))
df = pd.read_csv("sgRNA_design_9606_GRCh38_enAsCas12a_CRISPRko_NCBI_20200615.txt", sep='\t', skiprows=skip)

df

,Input,Quota,Target Taxon,Target Gene ID,Target Gene Symbol,Target Transcript,Target Alias,CRISPR Mechanism,Target Domain,Reference Sequence,...,On-Target Ruleset,On-Target Efficacy Score,On-Target Rank,Off-Target Rank,On-Target Rank Weight,Off-Target Rank Weight,Combined Rank,Pick Order,Picking Round,Picking Notes
0,1,MAX,9606,1,A1BG,NM_130786.4,NaN,CRISPRko,CDS,NC_000019.10,...,enPAM+GB,0.8171,13,29,1.0,1.0,1,1.0,1.0,NaN
1,1,MAX,9606,1,A1BG,NM_130786.4,NaN,CRISPRko,CDS,NC_000019.10,...,enPAM+GB,0.5417,160,338,1.0,1.0,231,99.0,2.0,Previously skipped due to: Spacing Violation: ...
2,1000,MAX,9606,1000,CDH2,NM_001792.5,NaN,CRISPRko,CDS,NC_000018.10,...,enPAM+GB,0.7255,188,978,1.0,1.0,586,885.0,4.0,Previously skipped due to: Outside Target Wind...
3,1000,MAX,9606,1000,CDH2,NM_001792.5,NaN,CRISPRko,CDS,NC_000018.10,...,enPAM+GB,0.4573,832,804,1.0,1.0,962,753.0,3.0,Previously skipped due to: Outside Target Wind...
4,10004,MAX,9606,10004,NAALADL1,NM_005468.3,NaN,CRISPRko,CDS,NC_000011.10,...,enPAM+GB,0.8208,17,364,1.0,1.0,82,57.0,2.0,Previously skipped due to: Spacing Violation: ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9990,MAX,9606,9990,SLC12A6,NM_005135.2,NaN,CRISPRko,CDS,NC_000015.10,...,enPAM+GB,0.4226,1053,774,1.0,1.0,1051,NaN,NaN,Spacing Violation: Too close to earlier pick a...
9996,9990,MAX,9606,9990,SLC12A6,NM_005135.2,NaN,CRISPRko,CDS,NC_000015.10,...,enPAM+GB,0.2962,1312,1140,1.0,1.0,1340,701.0,2.0,Previously skipped due to: Spacing Violation: ...
9997,9994,MAX,9606,9994,CASP8AP2,NM_012115.4,NaN,CRISPRko,CDS,NC_000006.12,...,enPAM+GB,0.7001,659,559,1.0,1.0,285,145.0,2.0,Previously skipped due to: Spacing Violation: ...
9998,9994,MAX,9606,9994,CASP8AP2,NM_012115.4,NaN,CRISPRko,CDS,NC_000006.12,...,enPAM+GB,0.6921,698,1292,1.0,1.0,811,411.0,2.0,Previously skipped due to: Spacing Violation: ...


## Exploratory

In [38]:
# 30bp-long sequences in context sequence col
df['sgRNA Context Sequence'].str.len().sort_values()
print("theyre all 34bp long")

theyre all 34bp long


In [42]:
# Number of sequences whose 25th-26th bp are 'GG'
np.count_nonzero(df['sgRNA Context Sequence'].str[25:27] == 'GG')
# result: theyre almost random, 10000 / 16 = ~625.

642

In [9]:
output = pd.DataFrame()
output['sgRNA_ID']= []
output['ntseq'] = []
output['phylo21'] = []
output['phast21'] = []
output['Pfam_domain'] = []
output['Pfam_family'] = []
output['distance'] = []
output['mod3'] = []
output['exlen'] = []
output['SD_distance'] = []
output['SA_distance'] = []
output['ntseq'] = []
output['AAcons7'] = []
output['AA_before_cut'] = []
output['AA_at_cut'] = []
output['AA_after_cut'] = []
output['Framshift_ratio_inDelphi'] = []
output['Doench2016'] = []
output['tracrv2'] = []
output


,sgRNA_ID,ntseq,phylo21,phast21,Pfam_domain,Pfam_family,distance,mod3,exlen,SD_distance,SA_distance,AAcons7,AA_before_cut,AA_at_cut,AA_after_cut,Framshift_ratio_inDelphi,Doench2016,tracrv2
